In [1]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import geopandas as gpd
import glob

In [2]:
city = gpd.read_file("../../data/irving_blocks/irving_city_blocks10.shp")
isd = gpd.read_file("../../data/irving_blocks/irving_isd_blocks10.shp")

In [3]:
irving_blocks = list(set(city.geoid) | set(isd.geoid) )

In [4]:
NUM_RUNS = 8
col_names = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 
             'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE',
             'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 
             'NHPIVAP', 'OTHERVAP', '2MOREVAP']
epsilon_values = [0.25, 0.5, 1, 2]
epsilon_splits = ["equal", "top_heavy", "mid_heavy", "bottom_heavy"]

In [5]:
irving_runs = pd.DataFrame()
for eps in epsilon_values:
    for split in epsilon_splits:
        df = pd.read_csv("results/noised_runs_{}_{}.csv".format(eps, split))
        df_test = df[df.GEOID.isin(irving_blocks)].pivot_table(index=["GEOID","epsilon","split","run"]).unstack()
        df_test.columns = ["{}_{}_noise".format(t[1], t[0]) for t in df_test.columns]
        irving_runs = irving_runs.append(df_test.reset_index())

In [6]:
irving_runs["GEOID"] = irving_runs["GEOID"].astype(str)

In [7]:
noised_isd = pd.merge(left=isd, right=irving_runs, left_on="geoid", right_on="GEOID")
noised_city = pd.merge(left=city, right=irving_runs, left_on="geoid", right_on="GEOID")

In [9]:
noised_isd.to_csv("irving_blocks/noised_isd.csv", index=False)
noised_city.to_csv("irving_blocks/noised_city.csv", index=False)

In [ ]:
noised_isd.to_file("irving_blocks/noised_isd.shp")
noised_city.to_file("irving_blocks/noised_city.shp")